# **Capstone 1 : AI Chatbot for Answer Evaluation**




# **1. Install required libraries**



*   `PyPDF2`: Used to extract text from PDF files.
*   `faiss-cpu`: A library for efficient similarity search using vector-based approaches (e.g., nearest neighbor search).
*   `transformers`: A popular library for handling pre-trained models (like GPT, T5, etc.).
*   `sentence-transformers`: Provides pre-trained models for generating sentence embeddings.
*   `langchain-community`: A library designed for building applications that utilize language models in a more flexible manner, integrating with various tools (like FAISS for indexing and retrieval).















In [1]:
# Install required libraries
!pip install PyPDF2 faiss-cpu transformers sentence-transformers
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.0 MB/s eta 0:00:00


# **2. Import required modules**

*   `PdfReader` from `PyPDF2` to read and extract text from the PDF file.
*   `AutoTokenizer`, `AutoModel`, and `AutoModelForSeq2SeqLM` from `transformers` for working with pre-trained transformer models (including tokenization and inference).
*   `torch`: PyTorch is used for model computations and managing tensors.
*   `numpy`: For numerical operations, although it’s not used explicitly in the code you provided.
*   `CharacterTextSplitter` from `langchain`: For splitting the raw extracted text into smaller, manageable chunks.
*   `FAISS`: For building a vector database for efficient similarity search.
*   `drive`: For mounting Google Drive to access files (in this case, the PDF file).
*   `HuggingFaceEmbeddings` is used to wrap the model and tokenizer into a single object for embedding generation in a FAISS index.
*   `The cosine_similarity` function from `sklearn.metrics.pairwise` is used to compute the cosine similarity between vectors. Cosine similarity measures the cosine of the angle between two non-zero vectors in a multidimensional space, giving a value between -1 and 1.













In [21]:
# Import required modules
from PyPDF2 import PdfReader
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
import torch
import numpy as np
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from google.colab import drive
from langchain.embeddings import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity

# **3. Mount Google Drive and load PDF**
This section mounts Google Drive to access files stored in Google Drive and sets the path to the target PDF file (UNIT-7 Optimizing E-commerce Systems NEW1.pdf).

In [3]:
# Mount Google Drive to access the PDF file
drive.mount('/content/drive')

# Path to the PDF file in Google Drive
pdf_path = '/content/drive/MyDrive/UNIT-7 Optimizing E-commerce Systems NEW1.pdf'

Mounted at /content/drive


# **4. Extract text from the PDF**



*   The `PdfReader` is used to read the PDF file.
*   Then, we loop through each page of the PDF and extract the text using `extract_text()`.
*   All the extracted text is combined into the `raw_text` variable.
*   Finally, the length of the extracted text is printed along with a preview of the first 1000 characters.







In [4]:
# Extract text from the PDF
reader = PdfReader(pdf_path)

raw_text = ''
for page in reader.pages:
    text = page.extract_text()
    if text:
        raw_text += text

print(f"Extracted Text Length: {len(raw_text)}")
print(raw_text[:1000])

Extracted Text Length: 41510
Outline:
❑Search Engine Optimization, 
❑Working mechanism of Search Engines, 
❑On Page SEO, Off Page SEO, Page Ranks, 
❑Using Google Analytics, Social Media Analytics, 
❑Recommendation Systems: Collaborative, Content Based, 
❑Use of Recommendation Systems in E -commerceUNIT 7: Optimizing E -Commerce Systems
12/28/2024 1UNIT -7 Optimizing E -Comemrce SystemSearch Engine Optimization (SEO)
12/28/2024 UNIT -7 Optimizing E -Comemrce System2The process of maximizing the number of visitors to a particular 
website by ensuring that the site appears high on the list of results 
returned by a search engine .
"the key to getting more traffic lies in integrating content with search 
engine optimization and social media marketing"
SEO stands for Search Engine Optimization and helps search engines 
understand your website’s content and connect it with users by 
delivering relevant, valuable results based on their search queries.
The goal of SEO is to rank on the fir

# **5. Split text into manageable chunks**



*   Here, the text is split into smaller chunks for better processing.

*   The `CharacterTextSplitter` is used to split the `raw_text`:

      `separator="\n"`: Chunks are split by newline characters.

      `chunk_size=500`: Each chunk will contain a maximum of 500 characters.

      `chunk_overlap=50`: Each chunk will overlap with the next by 50 characters (helps maintain context between chunks).

      The resulting chunks are stored in texts, and the number of chunks and a sample chunk are printed.



In [5]:
# Split the text into manageable chunks
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
)
texts = text_splitter.split_text(raw_text)

print(f"Number of text chunks: {len(texts)}")
print(f"Sample chunk:\n{texts[0]}")

Number of text chunks: 91
Sample chunk:
Outline:
❑Search Engine Optimization, 
❑Working mechanism of Search Engines, 
❑On Page SEO, Off Page SEO, Page Ranks, 
❑Using Google Analytics, Social Media Analytics, 
❑Recommendation Systems: Collaborative, Content Based, 
❑Use of Recommendation Systems in E -commerceUNIT 7: Optimizing E -Commerce Systems
12/28/2024 1UNIT -7 Optimizing E -Comemrce SystemSearch Engine Optimization (SEO)


# **6. Load a Hugging Face model for embeddings**

Here, we load a pre-trained model from Hugging Face for generating sentence embeddings.


*   `sentence-transformers/all-MiniLM-L6-v2` is a lightweight transformer model designed to produce dense vector embeddings for sentences.
*   `AutoTokenizer` and `AutoModel` are used to load the tokenizer and the model respectively.





In [6]:
# Load a Hugging Face model for embeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

# **7. Generate embeddings for text chunks**


*   This function takes the texts (split chunks of the PDF) and generates embeddings for each chunk using the pre-trained model.
*   The `tokenizer` converts each chunk into token IDs (as input for the model).
*   The model generates outputs, and we use `last_hidden_state` to get the embeddings for the input text. The embeddings are averaged (mean pooling) across all tokens in the chunk.
*   The `torch.no_grad()` context ensures that gradients are not calculated, saving memory and computation.



In [7]:
# Generate embeddings for text chunks
def get_embeddings(texts):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding="max_length")
        with torch.no_grad():
            outputs = model(**inputs)
            # Use mean pooling to create a single vector for each chunk
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
            embeddings.append(embedding)
    return embeddings

embeddings = get_embeddings(texts)

# **8. Create a FAISS index**


*   `FAISS.from_embeddings()` creates a `FAISS` index from the embeddings and text chunks.
*   This allows us to perform fast similarity search on the text chunks based on their embeddings.


In [8]:
# Create a FAISS index
text_embedding_pairs = list(zip(texts, embeddings))

embedding_function = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
)

# Pass the list of tuples and the embedding function to from_embeddings()
faiss_index = FAISS.from_embeddings(text_embedding_pairs, embedding_function)

<ipython-input-8-f7abf992d574>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# **9. Save the FAISS index for future use**
The `FAISS` index is saved to the local directory in Google Drive `/content/drive/MyDrive/faiss_index`. This makes it easier to load and reuse the index later without needing to re-generate it.

In [9]:
# Save the FAISS index for future use
faiss_index.save_local('/content/drive/MyDrive/faiss_index')

# **10. Load a pre-trained model for answering questions**
This loads a question-answering model `flan-t5-large` from Hugging Face. The model is specifically fine-tuned for tasks like text generation and answering questions based on provided context.

In [10]:
# Load a pre-trained model for answering questions
qa_model_name = "google/flan-t5-large"
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForSeq2SeqLM.from_pretrained(qa_model_name)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# **11. Defining the Core Function for Question Asking and Rating**

*   The `questions` list contains a set of predefined questions. These questions will be randomly selected for the chatbot to ask.
*   The function `np.random.choice(questions)` randomly selects a `question` from the questions list to ask the user.
*   `input()` is used to collect the user’s answer to the selected question.
*   Calling a prebuilt FAISS index (`faiss_index`) to find the most relevant content related to the current question. FAISS (Facebook AI Similarity Search) is a library that allows for fast similarity search, usually applied to retrieve text or documents most relevant to a given query.
*   The `relevant_chunks` contain retrieved results from the FAISS index. Each result is assumed to have a `page_content` attribute, which holds the text. These are concatenated together into a single string (`context`), which will serve as the context for generating the chatbot's answer.
*   The question and the context are formatted into a single string (`input_text`). The input is then tokenized using the `qa_tokenizer`. This step transforms the input text into a format that the model can understand.
*   Using `qa_model.generate()`, the answer is generated by the model based on the question and context. The `max_length`, `num_beams`, and
`early_stopping` parameters control the generation behavior (e.g., controlling length and encouraging diversity in the response). `torch.no_grad()` ensures that no gradients are calculated during this inference step (making it more efficient).
*   The generated output is decoded back into human-readable text using the `qa_tokenizer.decode()` method.
*   Both the user’s answer and the chatbot’s answer are passed through an embedding function (`embedding_function.embed_query()`). This converts each answer into a vector representation (embedding), which can be compared for similarity.
*   `cosine_similarity()` calculates how similar the two answer embeddings are. Cosine similarity ranges from -1 to 1, where 1 means identical vectors and -1 means completely opposite vectors. The similarity score is then flattened to extract the value and used for the next steps.
*   The similarity score is mapped to a rating between 1 and 5. The formula `min(max(int(similarity_score * 5), 1), 5`) scales the cosine similarity score (multiplies by 5) and ensures it falls within the range of 1 to 5.
*   The chatbot's generated answer, the user’s answer, and the similarity rating are printed for feedback. This gives the user a sense of how closely their answer matched the chatbot's generated answer.

In [24]:
# Function for Question Asking and Rating
def chatbot_ask_question_and_rate():

    # List of sample questions for the chatbot to ask
    questions = [
        "What is SEO?",
        "How is SEO different from SEM?",
        "What are the benefits of using recommendation systems in e-commerce?",
        "Explain the concept of collaborative filtering in recommendation systems.",
        "What is on-page SEO, and how does it differ from off-page SEO?",
        "How does Google Analytics help in digital marketing?",
        "What is the role of social media analytics in business decision-making?",
        "What are the critical steps involved in optimizing e-commerce systems?"
    ]

    # Pick a random question from the list
    question = np.random.choice(questions)
    print(f"Chatbot: {question}")

    # Get the user's answer
    user_answer = input("Your answer: ")

    # Retrieve top 5 relevant chunks from the index
    relevant_chunks = faiss_index.similarity_search(question, k=5)

    # Extract page content from relevant chunks
    context = " ".join([chunk.page_content for chunk in relevant_chunks])

    # Prepare the QA model input
    input_text = f"Question: {question}\nContext: {context}"
    inputs = qa_tokenizer(input_text, return_tensors="pt", truncation=True, max_length=1024)

    # Generate the answer using the model
    with torch.no_grad():
        outputs = qa_model.generate(**inputs, max_length=300, num_beams=5, early_stopping=True)
    chatbot_answer = qa_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Compare the user's answer with the chatbot's generated answer
    # Embed both the user's answer and the chatbot's answer for similarity evaluation
    user_answer_embedding = embedding_function.embed_query(user_answer)
    chatbot_answer_embedding = embedding_function.embed_query(chatbot_answer)

    # Calculate cosine similarity between user answer and chatbot answer
    similarity_score = cosine_similarity([user_answer_embedding], [chatbot_answer_embedding]).flatten()[0]

    # Rate the user's answer based on similarity (1 to 5 scale)
    rating = min(max(int(similarity_score * 5), 1), 5)

    # Provide feedback to the user
    print(f"\nChatbot's Answer: {chatbot_answer}")
    print(f"Your Answer: {user_answer}")
    print(f"Similarity Rating (1-5): {rating}")


# **12. Running the Chatbot Interaction in a Loop**

*   This loop continuously calls the `chatbot_ask_question_and_rate()` function, asking the user a question, receiving their answer, generating a chatbot response, comparing the answers, and providing feedback.
*   After each round, the user is asked whether they want to try again. If the user types anything other than `yes`, the loop ends and the program stops.


In [23]:
# Test the chatbot's question-asking and evaluation
while True:
    chatbot_ask_question_and_rate()
    continue_ = input("\nDo you want to try again? (yes/no): ")
    if continue_.lower() != 'yes':
        break

Chatbot: What are the benefits of using recommendation systems in e-commerce?
Your answer:  recommendation systems help create a more efficient, personalized, and profitable shopping environment for both customers and businesses.

Chatbot's Answer: Operational efficiency – Deep analysis of social media can help organizations improve how they gauge demand. Retailers and others can use that information to manage inventory and suppliers, reduce costs
Your Answer:  recommendation systems help create a more efficient, personalized, and profitable shopping environment for both customers and businesses.
Similarity Rating (1-5): 1

Do you want to try again? (yes/no): yes
Chatbot: What is the role of social media analytics in business decision-making?
Your answer: Social media analytics plays a crucial role in business decision-making by providing valuable insights into customer behavior, market trends, and brand performance. By analyzing data from platforms like Facebook, Instagram, Twitter, a

# **Conclusion:**

The code is a simple chatbot interaction where the chatbot asks a randomly selected question, the user provides an answer, and then the chatbot generates an answer. Both answers are compared, and a similarity rating is given based on how close the two answers are. The user receives feedback on how closely their answer matched the chatbot’s, including a similarity score on a scale of 1 to 5.